<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# Neural Collaborative Filtering on MovieLens dataset.

Neural Collaborative Filtering (NCF) is a well known recommendation algorithm that generalizes the matrix factorization problem with multi-layer perceptron. 

This notebook provides an example of how to utilize and evaluate NCF implementation in the `recommenders`. We use a smaller dataset in this example to run NCF efficiently with GPU acceleration on a [Data Science Virtual Machine](https://azure.microsoft.com/en-gb/services/virtual-machines/data-science-virtual-machines/).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.notebook_utils import store_metadata

/Users/jchang/miniforge3/envs/movies/lib/python3.12/site-packages/pandera/engines/pandas_engine.py:67: UserWarning: Using typeguard < 3. Generic types like List[TYPE], Dict[TYPE, TYPE] will only validate the first element in the collection.
  warnings.warn(


Set the default parameters.

In [3]:
# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'

# Model parameters
EPOCHS = 30
BATCH_SIZE = 256

SEED = 42

### 1. Download the MovieLens dataset

In [4]:
df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating", "timestamp"]
)

INFO:recommenders.datasets.download_utils:Downloading https://files.grouplens.org/datasets/movielens/ml-1m.zip
100%|██████████| 5.78k/5.78k [00:00<00:00, 13.4kKB/s]


### 2. Split the data using the Spark chronological splitter provided in utilities

In [5]:
train, test = python_chrono_split(df, 0.75)

Filter out any users or items in the test set that do not appear in the training set.

In [6]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

Write datasets to csv files.

In [7]:
train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

Generate an NCF dataset object from the data subsets.

In [8]:
data = NCFDataset(train_file=train_file, test_file=test_file, seed=SEED)

INFO:recommenders.models.ncf.dataset:Indexing ./train.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test.csv ...
INFO:recommenders.models.ncf.dataset:Indexing ./test_full.csv ...


### 3. Train the NCF model on the training data, and get the top-k recommendations for our testing data

NCF accepts implicit feedback and generates prospensity of items to be recommended to users in the scale of 0 to 1. A recommended item list can then be generated based on the scores. Note that this quickstart notebook is using a smaller number of epochs to reduce time for training. As a consequence, the model performance will be slighlty deteriorated. 

In [9]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

/Users/jchang/miniforge3/envs/movies/lib/python3.12/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
I0000 00:00:1745429636.149458  987064 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


In [10]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time))

INFO:recommenders.models.ncf.ncf_singlenode:Epoch 10 [17.42s]: train_loss = 0.270731 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 20 [17.98s]: train_loss = 0.264117 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 30 [14.88s]: train_loss = 0.260995 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 40 [16.25s]: train_loss = 0.259119 
INFO:recommenders.models.ncf.ncf_singlenode:Epoch 50 [16.47s]: train_loss = 0.258010 


Took 813.7538 seconds for training.


In the movie recommendation use case scenario, seen movies are not recommended to the users.

In [11]:
with Timer() as test_time:
    users, items, preds = [], [], []
    item = list(train.itemID.unique())
    for user in train.userID.unique():
        user = [user] * len(item) 
        users.extend(user)
        items.extend(item)
        preds.extend(list(model.predict(user, item, is_list=True)))

    all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

    merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
    all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

print("Took {} seconds for prediction.".format(test_time))

Took 21.3426 seconds for prediction.


### 4. Evaluate how well NCF performs

The ranking metrics are used for evaluation.

In [12]:
# Save the model
model.save('ncf_model.h5')

# Save all necessary data
with open('ncf_dataset.pkl', 'wb') as f:
    pickle.dump({
        'user_mapping': data.user2id,
        'item_mapping': data.item2id,
        'n_users': data.n_users,
        'n_items': data.n_items
    }, f)